In [1]:
import matplotlib.pyplot as plt
import geopandas
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import cartopy.crs as ccrs
import numpy as np

def scale_bar(ax, length=None, location=(0.5, 0.05), linewidth=3):
    """
    ax is the axes to draw the scalebar on.
    length is the length of the scalebar in km.
    location is center of the scalebar in axis coordinates.
    (ie. 0.5 is the middle of the plot)
    linewidth is the thickness of the scalebar.
    """
    #Get the limits of the axis in lat long
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    #Make tmc horizontally centred on the middle of the map,
    #vertically at scale bar location
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    #Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(tmc)
    #Turn the specified scalebar location into coordinates in metres
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    #Calculate a scale bar length if none has been given
    #(Theres probably a more pythonic way of rounding the number but this works)
    if not length: 
        length = (x1 - x0) / 5000 #in km
        ndim = int(np.floor(np.log10(length))) #number of digits in number
        length = round(length, -ndim) #round to 1sf
        #Returns numbers starting with the list
        def scale_number(x):
            if str(x)[0] in ['1', '2', '5']: return int(x)        
            else: return scale_number(x - 10 ** ndim)
        length = scale_number(length) 

    #Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbx - length * 500, sbx + length * 500]
    #Plot the scalebar
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    #Plot the scalebar label
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

In [4]:
municipios = geopandas.read_file('municipios/Municipios_IBGE.shp')

In [5]:
lista_municipio = ['Caxias do Sul', 'Santa Maria', 'Porto Alegre', 'Pelotas']
def plot_municipios(municipios, lista_municipio, ax, c='black', delta=0.05):
    x = []
    y = []
    for index, row in municipios.iterrows():
        for m in lista_municipio:
            if row['NOME'] == m.upper():
                y.append(row['LATITUDESE'])
                x.append(row['LONGITUDES'])
    ax.scatter(x=x, y=y, marker='x', c=c)
    for i, txt in enumerate(lista_municipio[::-1]):
        ax.annotate(txt, (x[i]+delta, y[i]+delta), c=c)

In [6]:
ocorr = geopandas.read_file('cprm/OcorrÛncias_minerais.shp')
ocorr = ocorr[ocorr['UF'] == 'RS']

In [7]:
ocorr['SUBSTANCIA'].unique()

array(['Ágata', 'Calcedônia, Quartzo hialino (Cristal de rocha), Ágata',
       'Calcedônia gemológica, Quartzo hialino (Cristal de rocha), Ágata, Ônix',
       'Calcedônia gemológica, Quartzo hialino (Cristal de rocha), Ágata',
       'Quartzo hialino (Cristal de rocha), Ágata',
       'Ametista, Quartzo hialino (Cristal de rocha), Ágata, Ônix',
       'Calcário', 'Estanho', 'Carvão mineral', 'Feldspato',
       'Madeira fossilizada', 'Cobre', 'Talco', 'Molibdênio', 'Ferro',
       'Ouro', 'Calcedônia, Jaspe',
       'Ametista, Quartzo hialino (Cristal de rocha), Ágata',
       'Calcedônia gemológica, Cálcio, Quartzo hialino (Cristal de rocha), Ágata',
       'Ametista, Citrino',
       'Cálcio, Quartzo hialino (Cristal de rocha), Ágata, Ônix',
       'Cálcio, Opala, Quartzo hialino (Cristal de rocha), Ágata',
       'Opala, Zeólita, Ágata', 'Turfa', 'Tungstênio', 'Chumbo',
       'Chumbo, Cobre', 'Flúor', 'Dolomito', 'Bário', 'Cromo', 'Argila',
       'Pirita', 'Brita', 'Saibro', 'Qu

In [8]:
grupos = {'Materiais para construção civil':['AREIA', 'AREIA DE FUNDIÇÃO', 'AREIA QUARTZOSA', 'AREIA COMUM', 'AREIA FLUVIAL', 'ARENITO', 'ARGILA', 'ARGILA REFRATÁRIA', 'ARGILITO', 'BASALTO', 'BASALTO P/ BRITA', 'BENTONITA', 'CALCÁRIO', 'CALCÁRIO CALCÍTICO', 'CASCALHO', 'DIABÁSIO', 'DIABÁSIO P/ BRITA', 'GABRO', 'GNAISSE', 'GRANITO', 'QUARTZITO', 'RIÓLITO', 'SAIBRO', 'CALCITA', 'GIPSITA'],
'Insumos agrícolas': ['CALCÁRIO', 'CALCÁRIO DOLOMÍTICO', 'CALCÁRIO CALCÍTICO', 'CALCÁRIO CONCHÍFERO', 'CONCHAS CALCÁRIAS', 'PEDRA CALCÁRIA', 'CAULIM', 'DOLOMITO', 'FOSFATO', 'TALCO'],
'Recursos energéticos': ['ANTRACITO', 'ARENITO BETUMINOSO', 'CARVÃO MINERAL', 'CARVÃO', 'FOLHELHO BETUMINOSO', 'FOLHELHO PIROBETUMINOSO', 'LINHITO', 'ROCHA BETUMINOSA', 'SAPROPELITO', 'TURFA'],
'Minerais metálicos': ['CASSITERITA', 'CHUMBO', 'COBRE', 'FERRO', 'ILMENITA', 'LIMONITA', 'MINÉRIO DE CHUMBO', 'MINÉRIO DE COBRE', 'MINÉRIO DE OURO', 'MINÉRIO DE TITÂNIO', 'MINÉRIO DE ZINCO', 'MOLIBDÊNIO', 'OURO', 'PRATA', 'TANTALITA', 'TITÂNIO', 'PIRITA', 'ESTANHO'],
'Indústria cerâmica': ['ARGILA', 'CALCÁRIO', 'ARGILA REFRATÁRIA', 'ARGILA VERMELHA', 'ARGILA CAULÍNICA', 'ARGILA P/CER. VERMELH', 'ARGILITO', 'BENTONITA', 'CAULIM', 'FILITO'],
'Rochas ornamentais': ['ANORTOSITO', 'ARENITO', 'BASALTO', 'BASALTO P/ REVESTIMENTO', 'DIABÁSIO', 'GRANITO', 'GRANITO ORNAMENTAL', 'MÁMORE', 'QUARTZITO', 'RIÓLITO', 'SIENITO', 'ROCHA ORNAMENTAL'],  
'Pedras preciosas':['ÁGATA', 'ÁGUA MARINHA', 'AMETISTA', 'CALCEDÔNIA', 'CITRINO', 'GEMA', 'OPALA', 'QUARTZO', 'QUARTZO HIALINO (CRISTAL DE ROCHA)',  'QUARTZO FUMÊ', 'QUARTZO ROSA', 'CALCEDÔNIA GEMOLÓGICA', 'JASPE', 'ÔNIX', 'ZIRCÔNIO', 'BERÍLIO'],
'Águas minerais': ['ÁGUA MINERAL ALC. BIC', 'ÁGUA MINERAL', 'ÁGUA POTÁVEL DE MESA']
         }

In [ ]:
for grupo in grupos:
    print(grupo)
    fig, ax = plt.subplots(figsize=(12,12), subplot_kw={'projection': ccrs.PlateCarree()})
    municipios.plot(ax=ax, color='gainsboro', edgecolor='silver')

    scale_bar(ax, 100, location=(0.5,0.01))

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle='--')

    #gl.xlabels_bottom = True

    #colors
    #colormap = plt.cm.Set1
    #colors = [colormap(i) for i in np.linspace(0, 1., len(gp['SUBS'].unique()))]
    colors = sns.color_palette("Set1", 11)
    colors = ["#89C5DA", "#DA5724", "#74D944", "#CE50CA", "#3F4921", "#C0717C", "#CBD588", "#5F7FC7", "#673770", "#D3D93E", "#38333E", "#508578", "#D7C1B1", "#689030", "#AD6F3B", "#CD9BCD", "#D14285", "#6DDE88", "#652926", "#7FDCC0", "#C84248", "#8569D5", "#5E738F", "#D1A33D", "#8A7C64", "#599861"]
    ax.set_prop_cycle('color', colors)
    
    #ocorr.plot(ax=ax, column='SUBSTANCIA', legend=True, legend_kwds=dict(bbox_to_anchor=(1.04, 1), edgecolor = 'white', fontsize='14', title='SUBSTÂNCIAS:', title_fontsize='14', loc='upper left'))

    for s in grupos[grupo]:
        print(s)
        xs = []
        ys = []
        for index, row in ocorr.iterrows():
            mins = row['SUBSTANCIA'].split(', ')
            mins = [a.upper() for a in mins]
            #print(mins)
            
            if s in mins:
                x = ocorr['geometry'][index].x
                y = ocorr['geometry'][index].y
                xs.append(x)
                ys.append(y)
                
            ax.scatter(xs, ys, label=s)
        
    x, y, arrow_length = 0.04, 0.12, 0.1
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
    arrowprops=dict(facecolor='black', width=5, headwidth=15),
    ha='center', va='center', fontsize=12,
    xycoords=ax.transAxes)

    plot_municipios(municipios, lista_municipio, ax, c='black', delta=0.05)

    ax.set_title('OCORRÊNCIAS MINERAIS - {}'.format(grupo.upper()), loc='center', fontsize='14')
    #fig.suptitle('Potencial - {}'.format(grupo), fontsize='14')
    #plt.legend(bbox_to_anchor=(1.04, 1), edgecolor = 'white', fontsize='14', title='SUBSTÂNCIAS:', title_fontsize='14', loc='upper left')

    #nome = mineral.replace('/', '')
    plt.savefig('imagens/ocorrencias/'+grupo+".png", bbox_inches='tight', facecolor='white', dpi = 300, bbox_extra_artists=([ax]))
    #plt.show()

Materiais para construção civil
AREIA
AREIA DE FUNDIÇÃO
AREIA QUARTZOSA
AREIA COMUM
AREIA FLUVIAL
ARENITO
ARGILA
ARGILA REFRATÁRIA
ARGILITO
BASALTO
BASALTO P/ BRITA
BENTONITA
CALCÁRIO
CALCÁRIO CALCÍTICO
CASCALHO
DIABÁSIO
DIABÁSIO P/ BRITA
GABRO
GNAISSE
GRANITO
QUARTZITO
RIÓLITO
SAIBRO
CALCITA
GIPSITA
Insumos agrícolas
CALCÁRIO
CALCÁRIO DOLOMÍTICO
CALCÁRIO CALCÍTICO
CALCÁRIO CONCHÍFERO
CONCHAS CALCÁRIAS
PEDRA CALCÁRIA
CAULIM
DOLOMITO
FOSFATO
TALCO
